In [9]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib, pymysql, calendar, time, json
from urllib.request import urlopen
from datetime import datetime
from threading import Timer

class DBUpdater:  
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
        self.conn = pymysql.connect(host='localhost', user='root',
            password='0484', db='INVESTAR', charset='utf8')
        
        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code))
            """
            curs.execute(sql)
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
            curs.execute(sql)
        self.conn.commit()
        self.codes = dict()
               
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close() 
     
    def read_krx_code(self):
        """KRX로부터 상장기업 목록 파일을 읽어와서 데이터프레임으로 반환"""
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
        krx = pd.read_html(url, header=0)[0]
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx
    
    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에 저장"""
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]
                    
        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone()
            today = datetime.today().strftime('%Y-%m-%d')
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
                krx = self.read_krx_code()
                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]                
                    sql = f"REPLACE INTO company_info (code, company, last"\
                        f"_update) VALUES ('{code}', '{company}', '{today}')"
                    curs.execute(sql)
                    self.codes[code] = company
                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info "\
                        f"VALUES ({code}, {company}, {today})")
                self.conn.commit()
                print('')              

    def read_naver(self, code, company, pages_to_fetch):
        """네이버에서 주식 시세를 읽어서 데이터프레임으로 반환"""
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            with urlopen(url) as doc:
                if doc is None:
                    return None
                html = BeautifulSoup(doc, "lxml")
                pgrr = html.find("td", class_="pgRR")
                if pgrr is None:
                    return None
                s = str(pgrr.a["href"]).split('=')
                lastpage = s[-1] 
            df = pd.DataFrame()
            pages = min(int(lastpage), pages_to_fetch)
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url, page)
                df = df.append(pd.read_html(pg_url, header=0)[0])
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
                    format(tmnow, company, code, page, pages), end="\r")
            df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
                ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
            df['date'] = df['date'].replace('.', '-')
            df = df.dropna()
            df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
                'diff', 'open', 'high', 'low', 'volume']].astype(int)
            df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]
        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df

    def replace_into_db(self, df, num, code, company):
        """네이버에서 읽어온 주식 시세를 DB에 REPLACE"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{code}', "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, "\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'\
                'price [OK]'.format(datetime.now().strftime('%Y-%m-%d'\
                ' %H:%M'), num+1, company, code, len(df)))

    def update_daily_price(self, pages_to_fetch):
        """KRX 상장법인의 주식 시세를 네이버로부터 읽어서 DB에 업데이트"""  
        for idx, code in enumerate(self.codes):
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue
            self.replace_into_db(df, idx, code, self.codes[code])            

    def execute_daily(self):
        """실행 즉시 및 매일 오후 다섯시에 daily_price 테이블 업데이트"""
        self.update_comp_info()
        
        try:
            with open('config.json', 'r') as in_file:
                config = json.load(in_file)
                pages_to_fetch = config['pages_to_fetch']
        except FileNotFoundError:
            with open('config.json', 'w') as out_file:
                pages_to_fetch = 100 
                config = {'pages_to_fetch': 1}
                json.dump(config, out_file)
        self.update_daily_price(pages_to_fetch)

        tmnow = datetime.now()
        lastday = calendar.monthrange(tmnow.year, tmnow.month)[1]
        if tmnow.month == 12 and tmnow.day == lastday:
            tmnext = tmnow.replace(year=tmnow.year+1, month=1, day=1,
                hour=17, minute=0, second=0)
        elif tmnow.day == lastday:
            tmnext = tmnow.replace(month=tmnow.month+1, day=1, hour=17,
                minute=0, second=0)
        else:
            tmnext = tmnow.replace(day=tmnow.day+1, hour=17, minute=0,
                second=0)   
        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds
        t = Timer(secs, self.execute_daily)
        print("Waiting for next update ({}) ... ".format(tmnext.strftime
            ('%Y-%m-%d %H:%M')))
        t.start()

if __name__ == '__main__':
    dbu = DBUpdater()
    dbu.execute_daily()

Exception ignored in: <function DBUpdater.__del__ at 0x00000261E415B048>
Traceback (most recent call last):
  File "<ipython-input-1-f08c1a269c71>", line 41, in __del__
AttributeError: 'DBUpdater' object has no attribute 'conn'
Exception ignored in: <function DBUpdater.__del__ at 0x00000261E7462C18>
Traceback (most recent call last):
  File "<ipython-input-2-1ba968cad6bd>", line 41, in __del__
AttributeError: 'DBUpdater' object has no attribute 'conn'


[2020-09-28 21:06] #0001 REPLACE INTO company_info VALUES (155660, DSR, 2020-09-28)
[2020-09-28 21:06] #0002 REPLACE INTO company_info VALUES (001250, GS글로벌, 2020-09-28)
[2020-09-28 21:06] #0003 REPLACE INTO company_info VALUES (294870, HDC현대산업개발, 2020-09-28)
[2020-09-28 21:06] #0004 REPLACE INTO company_info VALUES (011070, LG이노텍, 2020-09-28)
[2020-09-28 21:06] #0005 REPLACE INTO company_info VALUES (066570, LG전자, 2020-09-28)
[2020-09-28 21:06] #0006 REPLACE INTO company_info VALUES (037560, LG헬로비전, 2020-09-28)
[2020-09-28 21:06] #0007 REPLACE INTO company_info VALUES (010060, OCI, 2020-09-28)
[2020-09-28 21:06] #0008 REPLACE INTO company_info VALUES (100840, S&TC, 2020-09-28)
[2020-09-28 21:06] #0009 REPLACE INTO company_info VALUES (064960, S&T모티브, 2020-09-28)
[2020-09-28 21:06] #0010 REPLACE INTO company_info VALUES (096770, SK이노베이션, 2020-09-28)
[2020-09-28 21:06] #0011 REPLACE INTO company_info VALUES (011810, STX, 2020-09-28)
[2020-09-28 21:06] #0012 REPLACE INTO company_info VAL

[2020-09-28 21:06] #0706 REPLACE INTO company_info VALUES (079190, EMW, 2020-09-28)
[2020-09-28 21:06] #0707 REPLACE INTO company_info VALUES (299170, IBKS제10호스팩, 2020-09-28)
[2020-09-28 21:06] #0708 REPLACE INTO company_info VALUES (950170, JTC, 2020-09-28)
[2020-09-28 21:06] #0709 REPLACE INTO company_info VALUES (021320, KCC건설, 2020-09-28)
[2020-09-28 21:06] #0710 REPLACE INTO company_info VALUES (036670, KCI, 2020-09-28)
[2020-09-28 21:06] #0711 REPLACE INTO company_info VALUES (044180, KD, 2020-09-28)
[2020-09-28 21:06] #0712 REPLACE INTO company_info VALUES (151860, KG ETS, 2020-09-28)
[2020-09-28 21:06] #0713 REPLACE INTO company_info VALUES (046440, KG모빌리언스, 2020-09-28)
[2020-09-28 21:06] #0714 REPLACE INTO company_info VALUES (122450, KMH, 2020-09-28)
[2020-09-28 21:06] #0715 REPLACE INTO company_info VALUES (160550, NEW, 2020-09-28)
[2020-09-28 21:06] #0716 REPLACE INTO company_info VALUES (950110, SBI핀테크솔루션즈, 2020-09-28)
[2020-09-28 21:06] #0717 REPLACE INTO company_info VAL

[2020-09-28 21:06] #1119 REPLACE INTO company_info VALUES (095700, 제넥신, 2020-09-28)
[2020-09-28 21:06] #1120 REPLACE INTO company_info VALUES (072520, 제넨바이오, 2020-09-28)
[2020-09-28 21:06] #1121 REPLACE INTO company_info VALUES (122310, 제노레이, 2020-09-28)
[2020-09-28 21:06] #1122 REPLACE INTO company_info VALUES (159580, 제로투세븐, 2020-09-28)
[2020-09-28 21:06] #1123 REPLACE INTO company_info VALUES (033100, 제룡전기, 2020-09-28)
[2020-09-28 21:06] #1124 REPLACE INTO company_info VALUES (033320, 제이씨현시스템, 2020-09-28)
[2020-09-28 21:06] #1125 REPLACE INTO company_info VALUES (044060, 조광ILI, 2020-09-28)
[2020-09-28 21:06] #1126 REPLACE INTO company_info VALUES (094360, 칩스앤미디어, 2020-09-28)
[2020-09-28 21:06] #1127 REPLACE INTO company_info VALUES (307930, 컴퍼니케이, 2020-09-28)
[2020-09-28 21:06] #1128 REPLACE INTO company_info VALUES (192250, 케이사인, 2020-09-28)
[2020-09-28 21:06] #1129 REPLACE INTO company_info VALUES (115500, 케이씨에스, 2020-09-28)
[2020-09-28 21:06] #1130 REPLACE INTO company_info VALUE

[2020-09-28 21:06] #1450 REPLACE INTO company_info VALUES (166090, 하나머티리얼즈, 2020-09-28)
[2020-09-28 21:06] #1451 REPLACE INTO company_info VALUES (025550, 한국선재, 2020-09-28)
[2020-09-28 21:06] #1452 REPLACE INTO company_info VALUES (041460, 한국전자인증, 2020-09-28)
[2020-09-28 21:06] #1453 REPLACE INTO company_info VALUES (021650, 한국큐빅, 2020-09-28)
[2020-09-28 21:06] #1454 REPLACE INTO company_info VALUES (256150, 한독크린텍, 2020-09-28)
[2020-09-28 21:06] #1455 REPLACE INTO company_info VALUES (340440, 한화플러스제1호스팩, 2020-09-28)
[2020-09-28 21:06] #1456 REPLACE INTO company_info VALUES (016790, 현대사료, 2020-09-28)
[2020-09-28 21:06] #1457 REPLACE INTO company_info VALUES (092300, 현우산업, 2020-09-28)
[2020-09-28 21:06] #1458 REPLACE INTO company_info VALUES (064240, 홈캐스트, 2020-09-28)
[2020-09-28 21:06] #1459 REPLACE INTO company_info VALUES (200670, 휴메딕스, 2020-09-28)
[2020-09-28 21:06] #1460 REPLACE INTO company_info VALUES (024060, 흥구석유, 2020-09-28)
[2020-09-28 21:06] #1461 REPLACE INTO company_info VA

...

[2020-09-29 00:46] #1667 에스엔유 (080000) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 00:46] #1668 에스엠코어 (007820) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 00:46] #1669 에스티오 (098660) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 00:46] #1670 에이에프더블류 (312610) : 311 rows > REPLACE INTO daily_price [OK]
[2020-09-29 00:46] #1671 에이치엘비 (028300) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 00:46] #1672 에이테크솔루션 (071670) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 00:46] #1673 에코마이스터 (064510) : 627 rows > REPLACE INTO daily_price [OK]
[2020-09-29 00:47] #1674 에프엔씨엔터 (173940) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 00:47] #1675 엘비세미콘 (061970) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 00:47] #1676 엘이티 (297890) : 70 rows > REPLACE INTO daily_price [OK]
[2020-09-29 00:47] #1677 엠에스씨 (009780) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 00:47] #1678 엠젠플러스 (032790) : 1000 rows > REPLACE INTO daily_price 

[2020-09-29 01:11] #1864 한신공영 (004960) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:11] #1865 한익스프레스 (014130) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:11] #1866 한전기술 (052690) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:11] #1867 한화투자증권 (003530) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:12] #1868 현대글로비스 (086280) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:12] #1869 현대중공업지주 (267250) : 834 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:12] #1870 현대해상 (001450) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:12] #1871 화승알앤에이 (013520) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:12] #1872 효성중공업 (298040) : 546 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:12] #1873 효성티앤씨 (298020) : 546 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:12] #1874 GRT (900290) : 967 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:12] #1875 GV (045890) : 1000 rows > REPLACE INTO daily_price [OK

[2020-09-29 01:36] #2062 에프앤가이드 (064850) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:36] #2063 엔솔바이오사이언스 (140610) : 506 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:36] #2064 옐로페이 (179720) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:37] #2065 인터코스 (240340) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:37] #2066 질경이 (233990) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:37] #2067 크로넥스 (215570) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:37] #2068 파마리서치바이오 (217950) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:37] #2069 판도라티비 (202960) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:37] #2070 BYC (001460) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:37] #2071 DB손해보험 (005830) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:38] #2072 GS (078930) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:38] #2073 HMM (011200) : 1000 rows > REPLACE INTO daily_price [

[2020-09-29 01:50] #2160 화신 (010690) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:50] #2161 휴켐스 (069260) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:50] #2162 흥국화재 (000540) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:50] #2163 AP시스템 (265520) : 853 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:50] #2164 AP위성 (211270) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:50] #2165 OQP (078590) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:50] #2166 S&K폴리텍 (091340) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:51] #2167 SFA반도체 (036540) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:51] #2168 THE MIDONG (161570) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:51] #2169 YTN (040300) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:51] #2170 골프존뉴딘홀딩스 (121440) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 01:51] #2172 글로벌에스엠 (900070) : 1000 rows > REPLACE INTO daily_pric

[2020-09-29 02:14] #2358 홈센타홀딩스 (060560) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 02:14] #2359 힘스 (238490) : 784 rows > REPLACE INTO daily_price [OK]
[2020-09-29 02:14] #2360 TS트릴리온 (284610) : 680 rows > REPLACE INTO daily_price [OK]
[2020-09-29 02:14] #2361 관악산업 (076340) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 02:14] #2362 나라소프트 (288490) : 457 rows > REPLACE INTO daily_price [OK]
[2020-09-29 02:14] #2363 데이드림엔터 (348840) : 30 rows > REPLACE INTO daily_price [OK]
[2020-09-29 02:14] #2364 루켄테크놀러지스 (162120) : 781 rows > REPLACE INTO daily_price [OK]
[2020-09-29 02:15] #2365 메디젠휴먼케어 (236340) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 02:15] #2366 셀젠텍 (258250) : 932 rows > REPLACE INTO daily_price [OK]
[2020-09-29 02:15] #2367 아이피몬스터 (223220) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 02:15] #2368 에스케이씨에스 (224020) : 1000 rows > REPLACE INTO daily_price [OK]
[2020-09-29 02:15] #2369 에이펙스인텍 (207490) : 454 rows > REPLACE INTO daily_price

### 우리나라 약2400개의 기업의 기업정보와 2016년부터 주식 정보를 가져왔다.
* 가져온 것을 데이터베이스에 저장. (하이디, mariaDB)
* 'investar'라는 데이터베이스에 'company_info'와 'daily_price'd